### 우선 원래 t가 1일 때의 Astute RAG 코드

In [ ]:
import json

from astute_rag import (
    combine_passage, make_internal_passage, make_external_passage, 
    make_prompts, make_passage_source, finalize_answer,
    make_internal_passage_batch, finalize_answer_batch  # 배치 버전 추가
)
from prompt_template import P_GEN, P_CON, P_ANS
from langchain_core.prompts import PromptTemplate
from acc_prec import load_results, calculate_accuracy_by_dataset_with_astute_rag

dataset_sizes = {
    "popqa": 260,
    "nq": 260, 
    "triviaqa": 261,
    "bioasq": 261
}

with open("datasets/total_qa_sampled/qa_dataset_with_base_api_rag_2.json", "r") as f:
    q = json.load(f)

P_gen, P_con, P_ans = make_prompts(P_GEN, P_CON, P_ANS)

E = make_external_passage()

# 배치 처리로 Internal passage 생성 (batch_size 조절 가능, GPU 메모리에 따라)
I = make_internal_passage_batch(q, P_gen, llm, batch_size=32)

combine_passages = combine_passage(E, I)
passage_source = make_passage_source(combine_passages)

# 컨텍스트 미리 생성
contexts = []
questions = []
for i in range(len(q)):
    context = "\n\n".join([f"[{j+1}]\nsource: {doc['source']}, content: {doc['page_content']}" for j, doc in enumerate(combine_passages[i])])
    contexts.append(context)
    questions.append(q[i]['question'])

# 배치 처리로 최종 답변 생성
finalize_answers = finalize_answer_batch(llm, questions, contexts, batch_size=4)

results = load_results('./output/output_with_base_api_rag_2.jsonl')
acc_with_astute_rag = calculate_accuracy_by_dataset_with_astute_rag(results, dataset_sizes, finalize_answers)  
acc_with_astute_rag["overall"] = sum(acc_with_astute_rag.values()) / len(acc_with_astute_rag)

### 이 중에서 여기까지는 공통

In [ ]:
import json

from astute_rag import (
    combine_passage, make_internal_passage, make_external_passage, 
    make_prompts, make_passage_source, finalize_answer,
    make_internal_passage_batch, finalize_answer_batch  # 배치 버전 추가
)
from prompt_template import P_GEN, P_CON, P_ANS
from langchain_core.prompts import PromptTemplate
from acc_prec import load_results, calculate_accuracy_by_dataset_with_astute_rag

dataset_sizes = {
    "popqa": 260,
    "nq": 260, 
    "triviaqa": 261,
    "bioasq": 261
}

with open("datasets/total_qa_sampled/qa_dataset.json", "r") as f:
    q = json.load(f)

P_gen, P_con, P_ans = make_prompts(P_GEN, P_CON, P_ANS)

E = make_external_passage()

# 배치 처리로 Internal passage 생성 (batch_size 조절 가능, GPU 메모리에 따라)
I = make_internal_passage_batch(q, P_gen, llm, batch_size=32)

combine_passages = combine_passage(E, I)
passage_source = make_passage_source(combine_passages)

# 컨텍스트 미리 생성
contexts = []
questions = []
for i in range(len(q)):
    context = "\n\n".join([f"[{j+1}]\nsource: {doc['source']}, content: {doc['page_content']}" for j, doc in enumerate(combine_passages[i])])
    contexts.append(context)
    questions.append(q[i]['question'])

/home/user/anaconda3/envs/knowledge_conflict/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/total_qa_sampled/qa_dataset_with_base_api_rag_2.json'